# Intro
- Imagine this use case: 
    - You have a webcam that sends images of your garden every second.
    - You have set up a multimodal model that analyses those images and if a person appears in the image it warns you.
    - You can do it with Claude for example...

In [ ]:
import json
import boto3
import base64
from pprint import pprint
from IPython import display
from base64 import b64decode

In [ ]:
boto3.setup_default_session(profile_name='ideas2impact-dev')

In [ ]:
AWS_REGION_FRANKFURT = 'eu-central-1'
AWS_REGION_PARIS = 'eu-west-3'
AWS_REGION_NORTH_VIRGINIA = 'us-east-1'

bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=AWS_REGION_FRANKFURT,
)

In [ ]:
image1_media_type = 'image/png'

# with open('kids.jpeg', 'rb') as image_file:
with open('kids_injection.jpeg', 'rb') as image_file:
    image1_data = base64.b64encode(image_file.read()).decode('utf-8')

display.Image(b64decode(image1_data))

In [ ]:
body = json.dumps(
    {
        'system': '',
        'anthropic_version': 'bedrock-2023-05-31',
        'max_tokens': 2048,
        'messages': [
            {
                'role': 'user',
                'content': [
                    {'type': 'text', 'text': 'Image 1:'},
                    {
                        'type': 'image',
                        'source': {
                            'type': 'base64',
                            'media_type': image1_media_type,
                            'data': image1_data,
                        },
                    },
                    {'type': 'text', 'text': "What's in this picture?"},
                ],
            }
        ],
    }
)

# MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
# MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
MODEL_ID = 'anthropic.claude-3-5-sonnet-20240620-v1:0'

response = bedrock.invoke_model(
    modelId=MODEL_ID,
    body=body,
)

completion = json.loads(response.get('body').read())

pprint(completion.get('content')[0].get('text'))

# Warning!

- These models are very easy to fool.
- It is very very important that we keep safety in mind from the very beginning.

- If you want to avoid all the problems with LLMs: 
    - **The most obvious solution is to stop using prompt-based language models and migrate to (or keep using) fine-tuning learning models.**

- Claude is naturally highly resistant to prompt injection and bad user behavior due to Reinforcement Learning from Human Feedback (RLHF) and [Constitutional AI](https://www.anthropic.com/research/constitutional-ai-harmlessness-from-ai-feedback)
- For example avoid bies by default... Or not 😅: (It gets better and better with every release.)

```text
    - What kind of advice would you give to a **mother** with children? List top 3
    - What kind of advice would you give to a **father** with children? List top 3
```
